In [1]:
# %%capture
# !git clone https://github.com/trinhdz-dz/delivery-robots.git
# %cd delivery-robots/marl-delivery-master
# !pip install -r requirements.txt

# IQLA

IQLAgent khởi tạo, lựa chọn hành động, cập nhật Q-table và lưu/tải knowledge

In [2]:
import numpy as np
import pickle
import os
from collections import defaultdict
import random
import math
import copy

class IQLAgent:
    def __init__(self, agent_id, n_actions=5, alpha=0.2, gamma=0.99, epsilon=1):
        """
        Khởi tạo agent Q-learning độc lập
        
        Args:
            agent_id: ID của robot
            n_actions: Số lượng hành động có thể (5 hướng di chuyển: S, L, R, U, D)
            alpha: Tốc độ học
            gamma: Hệ số giảm giá
            epsilon: Xác suất khám phá
        """
        self.agent_id = agent_id
        self.n_actions = n_actions
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        
        # Sử dụng defaultdict để tiết kiệm bộ nhớ cho Q-table
        self.q_table = defaultdict(lambda: np.zeros(n_actions))
        
        # Ánh xạ từ hành động đến chỉ số
        self.action_to_idx = {'S': 0, 'L': 1, 'R': 2, 'U': 3, 'D': 4}
        self.idx_to_action = {0: 'S', 1: 'L', 2: 'R', 3: 'U', 4: 'D'}
        
        # Lưu trữ các trạng thái đã thăm để cập nhật
        self.visited_states = set()
        
    def get_state_representation(self, state, env, robot_id, radius=15):
        """
        Trích xuất đặc trưng trạng thái cho robot với ID robot_id
        Chỉ xem xét các gói hàng trong bán kính nhất định để giảm không gian trạng thái
        """
        # Vị trí hiện tại của robot
        robot_pos = (state['robots'][robot_id][0] - 1, state['robots'][robot_id][1] - 1)
        carrying = state['robots'][robot_id][2]
        
        # Tìm các gói hàng trong bán kính
        nearby_packages = []
        for pkg in state['packages']:
            pkg_id = pkg[0]
            start_pos = (pkg[1] - 1, pkg[2] - 1)
            target_pos = (pkg[3] - 1, pkg[4] - 1)
            
            # Chỉ xem xét các gói ở trạng thái chờ đợi
            start_dist = abs(start_pos[0] - robot_pos[0]) + abs(start_pos[1] - robot_pos[1])
            
            if start_dist <= radius:
                nearby_packages.append((pkg_id, start_pos, target_pos))
        
        # Sắp xếp gói hàng theo ID
        nearby_packages.sort()
        
        # Chọn tối đa 3 gói hàng gần nhất để giữ trạng thái nhỏ
        nearby_packages = nearby_packages[:3]
        
        # Mã hóa trạng thái
        state_code = (robot_pos, carrying)
        
        for pkg in nearby_packages:
            state_code += (pkg[0], pkg[1], pkg[2])
            
        return state_code
    
    def choose_action(self, state, available_actions=None, epsilon_override=None):
        """
        Chọn hành động dựa trên chiến lược epsilon-greedy với khuyến khích chuyển động.
        
        Thêm tham số epsilon_override để tạm thời ghi đè giá trị epsilon của agent.
        """
        if available_actions is None:
            available_actions = list(range(self.n_actions))

        # Dùng epsilon_override nếu được truyền, ngược lại dùng self.epsilon
        eps = self.epsilon if epsilon_override is None else epsilon_override

        if np.random.random() < eps:
            # Khám phá với khuyến khích chuyển động (không ưu tiên hành động "đứng yên")
            # Tạo trọng số cho mỗi hành động: đứng yên có trọng số thấp hơn
            weights = [0.2 if a == 0 else 1.0 for a in available_actions]
            # Chuẩn hóa trọng số
            weights = np.array(weights) / sum(weights)
            return np.random.choice(available_actions, p=weights)
        else:
            # Khai thác
            q_values = self.q_table[state]
            available_q = [q_values[a] for a in available_actions]
            max_q = max(available_q)
            best = [a for a, q in zip(available_actions, available_q) if q == max_q]
            
            # Nếu có nhiều hành động tốt nhất và một trong số đó là đứng yên,
            # ưu tiên các hành động khác
            if len(best) > 1 and 0 in best:
                non_stay_actions = [a for a in best if a != 0]
                if non_stay_actions:
                    return np.random.choice(non_stay_actions)
                
            return np.random.choice(best)
    
    def update_q_table(self, state, action, reward, next_state, done):
        """
        Cập nhật bảng Q dựa trên phương trình Bellman
        """
        # Thêm trạng thái này vào danh sách đã thăm
        self.visited_states.add(state)
        
        # Lấy giá trị Q hiện tại
        q_current = self.q_table[state][action]
        
        if done:
            # Nếu đã hoàn thành, không có trạng thái tiếp theo
            q_target = reward
        else:
            # Giá trị Q mới dựa trên phương trình Bellman
            q_target = reward + self.gamma * np.max(self.q_table[next_state])
        
        # Cập nhật giá trị Q
        self.q_table[state][action] = q_current + self.alpha * (q_target - q_current)
    
    def save_q_table(self, filename):
        """
        Lưu Q-table vào file
        """
        # Chỉ lưu các trạng thái đã thăm để tiết kiệm dung lượng
        q_table_to_save = {state: self.q_table[state] for state in self.visited_states}
        with open(filename, 'wb') as f:
            pickle.dump(q_table_to_save, f)
    
    def load_q_table(self, filename):
        """
        Tải Q-table từ file
        """
        if os.path.exists(filename):
            with open(filename, 'rb') as f:
                saved_q_table = pickle.load(f)
                
            # Chuyển đổi thành defaultdict
            for state, values in saved_q_table.items():
                self.q_table[state] = values
                self.visited_states.add(state)
            
            print(f"Đã tải Q-table cho agent {self.agent_id} với {len(self.visited_states)} trạng thái")
            return True
        return False

# PathSharer

Điều phối đường đi (path coordination) cho nhiều robot trên một lưới (grid), với hai chức năng chính:

Giải quyết xung đột di chuyển giữa các robot (ưu tiên ID nhỏ hơn)

Tính bước đi đầu tiên từ vị trí hiện tại đến đích, dùng thuật toán BFS

In [3]:
from collections import deque

class PathSharer:
    def __init__(self, n_robots, grid):
        """
        Lớp điều phối đường đi giữa các robot
        
        Args:
            n_robots: Số lượng robot
            grid: Bản đồ môi trường
        """
        self.n_robots = n_robots
        self.grid = grid
        self.n_rows = len(grid)
        self.n_cols = len(grid[0])
        
        # Lưu trữ vị trí hiện tại và dự định đi đến của các robot
        self.current_positions = {}
        self.intended_positions = {}
        
        # Ánh xạ hành động đến vector di chuyển
        self.action_to_move = {
            'S': (0, 0),
            'L': (0, -1),
            'R': (0, 1),
            'U': (-1, 0),
            'D': (1, 0)
        }
    
    def update_positions(self, robots_state):
        """
        Cập nhật vị trí hiện tại của các robot
        """
        for i, robot in enumerate(robots_state):
            self.current_positions[i] = (robot[0] - 1, robot[1] - 1)
    
    def is_valid_position(self, pos):
        """
        Kiểm tra xem một vị trí có hợp lệ không (trong bản đồ và không phải vật cản)
        """
        r, c = pos
        if r < 0 or r >= self.n_rows or c < 0 or c >= self.n_cols:
            return False
        return self.grid[r][c] == 0
    
    def resolve_conflicts(self, intended_actions):
        """
        Giải quyết xung đột giữa các robot
        Robot với ID nhỏ hơn được ưu tiên
        
        Args:
            intended_actions: Dictionary ánh xạ robot_id -> hành động dự kiến
        
        Returns:
            Dictionary ánh xạ robot_id -> hành động cuối cùng
        """
        # Tính toán vị trí dự định cho mỗi robot
        self.intended_positions = {}
        for robot_id, action in intended_actions.items():
            r, c = self.current_positions[robot_id]
            dr, dc = self.action_to_move[action]
            self.intended_positions[robot_id] = (r + dr, c + dc)
        
        # Giải quyết xung đột theo thứ tự ưu tiên (ID nhỏ hơn được ưu tiên)
        final_actions = {}
        occupied_positions = set()
        
        for robot_id in sorted(intended_actions.keys()):
            action = intended_actions[robot_id]
            intended_pos = self.intended_positions[robot_id]
            
            # Kiểm tra xem vị trí dự định có hợp lệ không
            if not self.is_valid_position(intended_pos):
                final_actions[robot_id] = 'S'  # Đứng yên nếu vị trí không hợp lệ
                continue
            
            # Kiểm tra xung đột với các robot có ưu tiên cao hơn
            if intended_pos in occupied_positions:
                final_actions[robot_id] = 'S'  # Đứng yên nếu có xung đột
            else:
                final_actions[robot_id] = action  # Giữ nguyên hành động nếu không có xung đột
                occupied_positions.add(intended_pos)
                
                # Vị trí hiện tại cũng cần được đánh dấu là bị chiếm
                occupied_positions.add(self.current_positions[robot_id])
        
        return final_actions
    
    
    def calculate_path_bfs(self, start, goal, blocked_positions=None):
        if blocked_positions is None:
            blocked_positions = set()

        if start == goal:          # đã ở đích
            return 'S'

        # BFS từ *start* về *goal*
        q = deque([(start, None)])     # (ô hiện tại, action đầu tiên rời start)
        visited = {start}

        # Các bước kèm action tương ứng
        directions = [(-1,0,'U'), (1,0,'D'), (0,-1,'L'), (0,1,'R')]

        while q:
            (r, c), first_act = q.popleft()

            if (r, c) == goal:
                return first_act       # bước đầu tiên

            for dr, dc, act in directions:
                nr, nc = r+dr, c+dc
                nxt = (nr, nc)
                if (0 <= nr < self.n_rows and 0 <= nc < self.n_cols
                        and self.grid[nr][nc] == 0
                        and nxt not in blocked_positions
                        and nxt not in visited):
                    visited.add(nxt)
                    q.append((nxt, act if first_act is None else first_act))

        return 'S'

In [4]:
class CBSPathFinder:
    def __init__(self, grid, max_time=100):
        self.grid = grid
        self.n_rows = len(grid)
        self.n_cols = len(grid[0])
        self.max_time = max_time
        
        # Ánh xạ hành động đến vector di chuyển
        self.action_to_move = {
            'S': (0, 0),
            'L': (0, -1),
            'R': (0, 1),
            'U': (-1, 0),
            'D': (1, 0)
        }
        self.move_to_action = {v: k for k, v in self.action_to_move.items()}
    
    def is_valid_position(self, pos):
        r, c = pos
        if r < 0 or r >= self.n_rows or c < 0 or c >= self.n_cols:
            return False
        return self.grid[r][c] == 0
    
    def get_neighbors(self, pos):
        r, c = pos
        neighbors = []
        for dr, dc in [(0, 0), (0, 1), (1, 0), (0, -1), (-1, 0)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < self.n_rows and 0 <= nc < self.n_cols and self.grid[nr][nc] == 0:
                neighbors.append((nr, nc))
        return neighbors
    
    def compute_path(self, start, goal, constraints=None):
        """Tìm đường đi từ start đến goal phù hợp với constraints"""
        if constraints is None:
            constraints = {}
        
        # A* search với constraints
        open_set = [(0, 0, start, [])]  # (f, g, pos, path)
        closed_set = set()
        
        while open_set:
            f, g, pos, path = heapq.heappop(open_set)
            
            if pos == goal:
                if not path:
                    return 'S'  # Đã ở tại đích
                return path[0]  # Trả về bước đầu tiên
            
            if (pos, len(path)) in closed_set:
                continue
                
            closed_set.add((pos, len(path)))
            
            for next_pos in self.get_neighbors(pos):
                # Kiểm tra constraint
                time_step = len(path)
                constraint_key = (next_pos, time_step + 1)
                if constraint_key in constraints:
                    continue
                
                # Tạo hướng đi mới
                move = (next_pos[0] - pos[0], next_pos[1] - pos[1])
                action = self.move_to_action.get(move, 'S')
                new_path = path + [action]
                
                # Tính f = g + h
                new_g = g + 1
                h = abs(next_pos[0] - goal[0]) + abs(next_pos[1] - goal[1])
                new_f = new_g + h
                
                heapq.heappush(open_set, (new_f, new_g, next_pos, new_path))
        
        # Không tìm thấy đường đi
        return 'S'  # Mặc định đứng yên
    
    def cbs(self, starts, goals):
        """CBS algorithm for multi-agent path finding"""
        import heapq
        
        # Khởi tạo
        root = {'cost': 0, 'constraints': {}, 'paths': {}, 'collisions': []}
        
        # Tìm đường đi cho từng agent mà không có constraints
        for i in range(len(starts)):
            root['paths'][i] = self.compute_path(starts[i], goals[i])
        
        # Xác định xung đột
        root['collisions'] = self.detect_collisions(root['paths'])
        root['cost'] = sum(len(path) for path in root['paths'].values())
        
        # Min-heap ưu tiên theo cost
        open_list = [root]
        
        while open_list:
            node = heapq.heappop(open_list)
            
            if not node['collisions']:
                # Tất cả đường đi không có xung đột
                return node['paths']
            
            # Chọn một xung đột để giải quyết
            collision = node['collisions'][0]
            
            # Tạo hai conflict constraints
            for agent in collision['agents']:
                # Tạo một ràng buộc mới cho agent
                new_constraints = node['constraints'].copy()
                if agent not in new_constraints:
                    new_constraints[agent] = set()
                
                pos, time = collision['pos'], collision['time']
                new_constraints[agent].add((pos, time))
                
                # Tạo node mới
                new_node = {
                    'constraints': new_constraints,
                    'paths': node['paths'].copy(),
                    'collisions': []
                }
                
                # Tính lại đường đi cho agent bị ảnh hưởng
                new_node['paths'][agent] = self.compute_path(
                    starts[agent], goals[agent], new_constraints[agent])
                
                # Cập nhật collisions và cost
                new_node['collisions'] = self.detect_collisions(new_node['paths'])
                new_node['cost'] = sum(len(path) for path in new_node['paths'].values())
                
                heapq.heappush(open_list, new_node)
        
        # Mặc định: mỗi agent đứng yên
        return {i: 'S' for i in range(len(starts))}
    
    def detect_collisions(self, paths):
        """Phát hiện xung đột trong các đường đi"""
        collisions = []
        
        # Kiểm tra xung đột ở nút
        agent_positions = {}
        for agent, path in paths.items():
            if not path:
                continue
                
            current_pos = None  # TODO: Cần lấy vị trí hiện tại của agent
            
            for t, action in enumerate(path):
                dr, dc = self.action_to_move[action]
                next_pos = (current_pos[0] + dr, current_pos[1] + dc)
                
                if (next_pos, t+1) in agent_positions:
                    other_agent = agent_positions[(next_pos, t+1)]
                    collisions.append({
                        'agents': [agent, other_agent],
                        'pos': next_pos,
                        'time': t+1
                    })
                
                agent_positions[(next_pos, t+1)] = agent
                current_pos = next_pos
        
        return collisions

# MultiAgent

Đa tác tử

In [5]:
class MultiAgentDelivery:
    def __init__(self, env, use_trained_model=True):
        """
        Lớp chính để quản lý đa tác tử IQL
        
        Args:
            env: Môi trường giao hàng
            use_trained_model: Có sử dụng mô hình đã huấn luyện không
        """
        self.env = env
        self.n_robots = len(env.robots)
        self.grid = env.grid
        
        # Khởi tạo các IQL agents và PathSharer
        self.agents = [IQLAgent(i) for i in range(self.n_robots)]
        self.path_sharer = PathSharer(self.n_robots, self.grid)
        
        # Trạng thái gói hàng
        self.packages_status = {}  # package_id -> status
        
        # Tải mô hình đã huấn luyện (nếu được yêu cầu)
        if use_trained_model:
            self.load_models()
    
    def load_models(self, folder="models"):
        """
        Tải các mô hình Q-learning đã được huấn luyện
        """
        if not os.path.exists(folder):
            os.makedirs(folder)
            
        for agent in self.agents:
            agent.load_q_table(f"{folder}/agent_{agent.agent_id}.pkl")
    
    def save_models(self, folder="models"):
        """
        Lưu các mô hình Q-learning đã huấn luyện
        """
        if not os.path.exists(folder):
            os.makedirs(folder)
            
        for agent in self.agents:
            agent.save_q_table(f"{folder}/agent_{agent.agent_id}.pkl")
    
    def update_package_status(self, state):
        """
        Cập nhật trạng thái gói hàng dựa trên thông tin từ môi trường.
        - Gói hàng đang được robot mang: 'in_transit'
        - Gói hàng hiện có trong môi trường nhưng không được robot mang: 'waiting'
        - Gói hàng không còn trong môi trường: loại bỏ khỏi packages_status
        """
        # 1. Lấy danh sách ID của các gói trong môi trường hiện tại
        current_package_ids = {pkg[0] for pkg in state['packages']}
        
        # 2. Xóa trạng thái của các gói không còn trong môi trường
        package_ids_to_remove = set(self.packages_status.keys()) - current_package_ids
        for pkg_id in package_ids_to_remove:
            del self.packages_status[pkg_id]
        
        # 3. Lấy danh sách ID các gói đang được robot mang
        carried_packages = {robot[2] for robot in state['robots'] if robot[2] > 0}
        
        # 4. Cập nhật trạng thái các gói hàng dựa trên tình trạng hiện tại
        for pkg in state['packages']:
            pkg_id = pkg[0]
            if pkg_id in carried_packages:
                self.packages_status[pkg_id] = 'in_transit'
            else:
                self.packages_status[pkg_id] = 'waiting'  # Đây là thay đổi quan trọng
        
    def is_package_available(self, pkg_id):
        """
        Kiểm tra xem gói hàng có sẵn sàng để nhận không
        """
        return pkg_id in self.packages_status and self.packages_status[pkg_id] == 'waiting'
    
    def find_best_package_for_robot(self, state, robot_id):
        """
        Tìm gói hàng tốt nhất cho robot dựa trên khoảng cách và thời hạn
        """
        robot_pos = (state['robots'][robot_id][0] - 1, state['robots'][robot_id][1] - 1)
        carrying = state['robots'][robot_id][2]
        
        # Nếu robot đã mang gói hàng, trả về gói hàng đó
        if carrying > 0:
            for pkg in state['packages']:
                if pkg[0] == carrying:
                    return carrying, (pkg[3] - 1, pkg[4] - 1)  # Trả về ID và vị trí đích
            
            # Nếu không tìm thấy gói hàng trong danh sách hiện tại, tìm trong lịch sử
            for pkg_id, status in self.packages_status.items():
                if pkg_id == carrying and status == 'in_transit':
                    # Đây là một giả định - trong thực tế bạn cần lưu thông tin đích của gói hàng
                    return carrying, None
        
        # Nếu robot không mang gói hàng, tìm gói hàng tốt nhất
        best_pkg = None
        best_score = float('inf')
        
        for pkg in state['packages']:
            pkg_id = pkg[0]
            start_pos = (pkg[1] - 1, pkg[2] - 1)
            deadline = pkg[6]
            
            # Chỉ xem xét các gói hàng đang chờ
            if not self.is_package_available(pkg_id):
                continue

            #kiểm tra xem gói hàng có trong trạng thái chờ không
            # print("Trạng thái gói hàng",self.is_package_available(pkg_id))
            
            # Tính khoảng cách Manhattan
            distance = abs(start_pos[0] - robot_pos[0]) + abs(start_pos[1] - robot_pos[1])
            
            # Tính điểm dựa trên khoảng cách và thời hạn
            # Tính điểm dựa trên khoảng cách và thời hạn
            # Gói hàng gần và có thời hạn sớm được ưu tiên
            urgency = max(0, deadline - state['time_step'] - distance)
            # Gói có deadline gần sẽ được ưu tiên hơn
            urgency_weight = 0.3  # Tăng trọng số cho tính khẩn cấp
            score = distance - urgency_weight * urgency

            # Thêm một phần thưởng nhỏ cho việc tìm gói hàng để khuyến khích robot di chuyển
            exploration_bonus = -0.5  # Khuyến khích tìm kiếm

            if score < best_score:
                best_score = score + exploration_bonus
                best_pkg = (pkg_id, start_pos)
                
        if best_pkg is not None:
            return best_pkg[0], best_pkg[1]
        else:
            # Nếu không tìm được gói phù hợp
            return None, None
        

    def shaping_reward(self, carrying, next_carrying, robot_pos, next_pos, goal_pos, robot_id=None, step_count=None, state=None):
        """
        Cơ chế phần thưởng định hình cho các robot
        """
        bonus = 0.0
        is_standing_still = (robot_pos == next_pos)
        
        # Khởi tạo từ điển theo dõi thời gian mang hàng nếu chưa có
        if not hasattr(self, 'package_carrying_steps'):
            self.package_carrying_steps = {i: {} for i in range(self.n_robots)}  # robot_id -> {pkg_id: steps}
        
        # 1) Thưởng nhận gói hàng
        if carrying == 0 and next_carrying > 0:
            bonus += 1.5
            # Bắt đầu đếm thời gian mang gói hàng
            if robot_id is not None:
                self.package_carrying_steps[robot_id][next_carrying] = 0
        
        # 2) Thưởng lớn khi thả gói hàng thành công
        if carrying > 0 and next_carrying == 0:
            bonus += 3.0
            # Reset bộ đếm thời gian mang hàng
            if robot_id is not None and carrying in self.package_carrying_steps[robot_id]:
                del self.package_carrying_steps[robot_id][carrying]
        
        # 3) Xử lý tình huống đang mang gói hàng
        if carrying > 0:
            # Cập nhật bộ đếm thời gian mang hàng
            if robot_id is not None:
                if carrying not in self.package_carrying_steps[robot_id]:
                    self.package_carrying_steps[robot_id][carrying] = 0
                self.package_carrying_steps[robot_id][carrying] += 1
                
                # Phạt tích lũy cho việc mang hàng quá lâu
                carrying_steps = self.package_carrying_steps[robot_id][carrying]
                
                # Bắt đầu phạt sau 20 bước
                if carrying_steps > 15:
                    # Phạt tăng dần theo thời gian, nhưng với tốc độ chậm hơn
                    time_penalty = 0.01 * (carrying_steps - 15)
                    # Giới hạn penalty tối đa
                    time_penalty = min(time_penalty, 1.5)
                    bonus -= time_penalty
                    
                    # Phạt nặng hơn nếu cả đứng yên và mang hàng quá lâu
                    if is_standing_still and carrying_steps > 20:
                        bonus -= 0.05 * (carrying_steps - 20)
            
            if goal_pos:
                # Tính khoảng cách Manhattan
                cur_dist = abs(robot_pos[0] - goal_pos[0]) + abs(robot_pos[1] - goal_pos[1])
                new_dist = abs(next_pos[0] - goal_pos[0]) + abs(next_pos[1] - goal_pos[1])
                
                if new_dist < cur_dist:
                    # Thưởng khi tiến gần đích (càng nhiều khi càng gần)
                    progress = cur_dist - new_dist
                    bonus += 0.7 * progress
                elif new_dist > cur_dist:
                    # Phạt nghiêm trọng khi đi xa đích khi đang mang hàng
                    regression = new_dist - cur_dist
                    bonus -= 0.9 * regression
                
                # Phạt thêm khi đứng yên và chưa đến đích
                if is_standing_still and cur_dist > 0:
                    bonus -= 0.5
            else:
                # Không biết đích ở đâu nhưng vẫn mang hàng, khuyến khích di chuyển
                if is_standing_still:
                    bonus -= 0.3
        
        # 4) Xử lý tình huống không mang gói hàng
        elif carrying == 0 and state is not None:
            # Tìm gói hàng tốt nhất cho robot này
            target_pkg_id, target_pkg_pos = self.find_best_package_for_robot(state, robot_id)
            
            if target_pkg_pos:  # Nếu có gói hàng phù hợp
                # Khoảng cách đến gói hàng gần nhất
                old_dist = abs(robot_pos[0] - target_pkg_pos[0]) + abs(robot_pos[1] - target_pkg_pos[1])
                new_dist = abs(next_pos[0] - target_pkg_pos[0]) + abs(next_pos[1] - target_pkg_pos[1])
                
                if new_dist < old_dist:
                    # Thưởng khi tiến gần gói hàng
                    bonus += 0.4 * (old_dist - new_dist)
                elif new_dist > old_dist:
                    # Phạt nhẹ khi đi xa gói hàng
                    bonus -= 0.2 * (new_dist - old_dist)
                
                # Phạt đứng yên khi có gói hàng gần đó
                if is_standing_still:
                    bonus -= 0.3
            elif is_standing_still and len(self.packages_status) > 0:
                # Phạt đứng yên khi còn gói hàng chưa giao
                bonus -= 0.1
        
        # 5) Phạt tích lũy nếu robot đi vòng quanh vô nghĩa (như đã đề xuất trước đó)
        if hasattr(self, 'position_history') and robot_id is not None:
            # Khởi tạo lịch sử vị trí nếu chưa có
            if not hasattr(self, 'position_history'):
                self.position_history = {i: [] for i in range(self.n_robots)}
                self.standing_still_count = {i: 0 for i in range(self.n_robots)}
            
            # Theo dõi vị trí lặp lại
            if len(self.position_history[robot_id]) > 10:
                # Đếm số lần xuất hiện của vị trí mới trong 10 bước gần đây
                recent_positions = self.position_history[robot_id][-10:]
                repeat_count = recent_positions.count(next_pos)
                
                if repeat_count >= 3:  # Nếu vị trí lặp lại nhiều lần
                    bonus -= 0.1 * repeat_count  # Phạt tăng dần
            
            # Theo dõi đứng yên
            if is_standing_still:
                self.standing_still_count[robot_id] += 1
                if self.standing_still_count[robot_id] > 3:  # Đứng yên quá 3 bước
                    bonus -= 0.1 * self.standing_still_count[robot_id]  # Phạt tăng dần
            else:
                self.standing_still_count[robot_id] = 0
                
            # Cập nhật lịch sử vị trí
            self.position_history[robot_id].append(next_pos)
            if len(self.position_history[robot_id]) > 20:  # Giới hạn lịch sử
                self.position_history[robot_id].pop(0)
        
        return bonus


    
    def train(self, n_episodes=1000, max_steps=1000):
        """
        Huấn luyện các agent sử dụng IQL
        """
        epsilon_decay = 0.995
        min_epsilon = 0.05
        
        #  Tạo biến để theo dõi tỷ lệ sử dụng BFS vs Q-Learning
        bfs_ratio = 0.9  # Ban đầu 90% sử dụng BFS, 10% sử dụng Q-Learning
        bfs_decay = 0.995  # Giảm dần tỷ lệ BFS sau mỗi episode
        
        for episode in range(n_episodes):
            state = self.env.reset()
            self.path_sharer.update_positions(state['robots'])
            self.update_package_status(state)
            
            total_reward = 0
            q_actions_count = 0  # Đếm số lần sử dụng Q-Learning
            bfs_actions_count = 0  # Đếm số lần sử dụng BFS
            
            for step in range(max_steps):
                intended_actions = {}
                
                for i, agent in enumerate(self.agents):
                    robot = state['robots'][i]
                    carrying = robot[2]
                    robot_pos = (robot[0] - 1, robot[1] - 1)
                    
                    # Quyết định có sử dụng BFS hay không dựa trên bfs_ratio
                    use_bfs = np.random.random() < bfs_ratio
                    
                    # Luôn lấy state_features để cập nhật Q-table
                    state_features = agent.get_state_representation(state, self.env, i)
                    
                    # THAY ĐỔI LỚN: Lựa chọn giữa BFS và Q-Learning
                    if use_bfs:
                        # Logic BFS giữ nguyên
                        target_pkg_id, target_pos = self.find_best_package_for_robot(state, i)
                        # print("kiểm tra gói hàng",target_pkg_id, "và vị trí",target_pos)
                        
                        if carrying > 0:
                            for pkg in state['packages']:
                                if pkg[0] == carrying:
                                    target_pos = (pkg[3] - 1, pkg[4] - 1)
                                    break
                            
                            if target_pos:
                                blocked_positions = {self.path_sharer.current_positions[j] 
                                                for j in range(self.n_robots) if j != i}
                                action = self.path_sharer.calculate_path_bfs(robot_pos, target_pos, blocked_positions)
                                intended_actions[i] = action
                                bfs_actions_count += 1
                            else:
                                # Fallback to Q-Learning
                                action_idx = agent.choose_action(state_features)
                                intended_actions[i] = agent.idx_to_action[action_idx]
                                q_actions_count += 1
                        elif target_pkg_id is not None:
                            blocked = {self.path_sharer.current_positions[j] 
                                    for j in range(self.n_robots) if j != i}
                            action = self.path_sharer.calculate_path_bfs(robot_pos, target_pos, blocked)
                            intended_actions[i] = action
                            bfs_actions_count += 1
                        else:
                            # Các trường hợp còn lại (tìm gói gần nhất)
                            waiting_pkgs = [((pkg[1]-1, pkg[2]-1), pkg[0]) 
                                        for pkg in state['packages']
                                        if self.is_package_available(pkg[0])]
                            
                            if waiting_pkgs:
                                nearest_pos, _ = min(waiting_pkgs,
                                                key=lambda x: abs(x[0][0]-robot_pos[0]) + abs(x[0][1]-robot_pos[1]))
                                blocked = {self.path_sharer.current_positions[j] 
                                        for j in range(self.n_robots) if j != i}
                                action = self.path_sharer.calculate_path_bfs(robot_pos, nearest_pos, blocked)
                                intended_actions[i] = action
                                bfs_actions_count += 1
                            else:
                                # Không còn gói, dùng Q-Learning
                                action_idx = agent.choose_action(state_features)
                                intended_actions[i] = agent.idx_to_action[action_idx]
                                q_actions_count += 1
                    else:
                        # Sử dụng Q-Learning để chọn hành động
                        action_idx = agent.choose_action(state_features)
                        intended_actions[i] = agent.idx_to_action[action_idx]
                        q_actions_count += 1
                

                # # In ra các hành động dự kiến của các robot
                # if (episode % 50 == 0) and (step % 50 == 0):  # In cứ mỗi 10 episode và mỗi 10 bước
                #     print(f"Episode {episode}, Step {step}, Intended Actions: {intended_actions}")
                
                # Giải quyết xung đột bằng PathSharer
                final_actions = self.path_sharer.resolve_conflicts(intended_actions)
                
                # # In ra các hành động sau khi giải quyết xung đột
                # if (episode % 50 == 0) and (step % 50 == 0):
                #     print(f"Episode {episode}, Step {step}, Final Actions: {final_actions}")
                

                # Xác định hành động nhận/thả gói hàng
                package_actions = ['0'] * self.n_robots
                
                
                for i in range(self.n_robots):
                    robot = state['robots'][i]
                    carrying = robot[2]
                    robot_pos = (robot[0] - 1, robot[1] - 1)
                    
                    if carrying > 0:
                        # Robot đang mang gói hàng, kiểm tra xem có ở đích không
                        target_found = False
                        for pkg in state['packages']:
                            if pkg[0] == carrying:

                                # Nếu gói hàng đã được tìm thấy, kiểm tra xem robot có ở đích không
                                # Đích của gói hàng
                                # print("Trạng thái robots",state['robots'])
                                # print("Trạng thái gói hàng",pkg)

                                target_pos = (pkg[3] - 1, pkg[4] - 1)
                                if robot_pos == target_pos:
                                    # print("Robot đã đến đích")
                                    package_actions[i] = '2'  # Thả gói hàng
                                    final_actions[i]  = 'S'
                                    target_found = True
                                break
                        
                        # Nếu không tìm thấy gói hàng trong danh sách hiện tại, giả định ta đã đến đích
                        if not target_found and final_actions[i] == 'S':
                            package_actions[i] = '2'  # Thử thả gói hàng
                    else:
                        # Robot không mang gói hàng, kiểm tra xem có gói hàng nào để nhận không
                        for pkg in state['packages']:
                            pkg_id = pkg[0]
                            start_pos = (pkg[1] - 1, pkg[2] - 1)
                            
                            if self.is_package_available(pkg_id) and robot_pos == start_pos:
                                package_actions[i] = '1'  # Nhận gói hàng
                                break

                
                
                # Kết hợp hành động di chuyển và hành động gói hàng
                combined_actions = [(final_actions[i], package_actions[i]) for i in range(self.n_robots)]
                
                # Thực hiện hành động và nhận phản hồi từ môi trường
                next_state, reward, done, info = self.env.step(combined_actions)
                                
                # Cập nhật vị trí robot và trạng thái gói hàng
                self.path_sharer.update_positions(next_state['robots'])
                self.update_package_status(next_state)

                # Kiểm tra xem còn gói hàng nào đang chờ không
                waiting_packages_exist = False
                for _, status in self.packages_status.items():
                    if status == 'waiting':
                        waiting_packages_exist = True
                        break
                                
                # Cập nhật bảng Q cho mỗi agent
                for i, agent in enumerate(self.agents):
                    state_features = agent.get_state_representation(state, self.env, i)
                    next_state_features = agent.get_state_representation(next_state, self.env, i)
                    
                    # Ánh xạ hành động thành chỉ số
                    action_idx = agent.action_to_idx[final_actions[i]]
                    
                    # Reward shaping như bình thường
                    next_robot = next_state['robots'][i]
                    next_carrying = next_robot[2]
                    next_pos = (next_robot[0]-1, next_robot[1]-1)
                    robot = state['robots'][i]
                    carrying = robot[2]
                    robot_pos = (robot[0]-1, robot[1]-1)
                    
                    # Xác định goal_pos
                    if carrying > 0:
                        goal_pkg = next((pkg for pkg in state['packages'] if pkg[0]==carrying), None)
                        if goal_pkg:
                            goal_pos = (goal_pkg[3]-1, goal_pkg[4]-1)
                        else:
                            goal_pos = None
                    else:
                        goal_pos = None
                    
                    
                    shaped = self.shaping_reward(
                        carrying, next_carrying, robot_pos, next_pos, goal_pos, 
                        robot_id=i, step_count=step, state=state
                    )
                    agent_reward = reward + shaped
                    
                    # Cập nhật Q-table với hành động từ BFS
                    agent.update_q_table(state_features, action_idx, agent_reward, next_state_features, done)
            
                #In ra 20 bước đầu tiên
                # if step < 40:
                #     print("==========================")
                #     self.env.render()
                #     print("Đích của các gói hàng: {pkg}".format(pkg=state['packages']))
                #     print(f"Hành động đã thực hiện: {combined_actions}")
                #     print(f"Phần thưởng: {reward}")
                
                # Cập nhật trạng thái và tổng phần thưởng
                state = next_state
                total_reward += reward
                
                if done:
                    break
            
            bfs_ratio = max(bfs_ratio * bfs_decay, 0.1)  # Giảm dần đến tối thiểu 10% sử dụng BFS
        
            for agent in self.agents:
                agent.epsilon = max(agent.epsilon * epsilon_decay, min_epsilon)
            
            # In thông tin huấn luyện
            if (episode + 1) % 10 == 0:
                print(f"Episode {episode + 1}/{n_episodes}, Total Reward: {total_reward}, Steps: {step + 1}, Epsilon: {self.agents[0].epsilon:.4f}")
                total_states = sum(len(agent.visited_states) for agent in self.agents)
                print(f"Episode {episode + 1}, Total states visited: {total_states}")
                print(f"Epsilon: {self.agents[0].epsilon:.4f}, BFS ratio: {bfs_ratio:.4f}")
                print(f"Q-Learning actions: {q_actions_count}, BFS actions: {bfs_actions_count}")
                print(f"Total states visited: {sum(len(agent.visited_states) for agent in self.agents)}")
                    
                # Kiểm tra giá trị Q cho một số trạng thái ngẫu nhiên
                # if total_states > 0:
                #     for i, agent in enumerate(self.agents):
                #         if agent.visited_states:
                #             random_state = random.choice(list(agent.visited_states))
                #             q_values = agent.q_table[random_state]
                #             print(f"Agent {i}, Sample Q-values: {q_values}")

            # Lưu mô hình sau mỗi 100 tập
            if (episode + 1) % 100 == 0:
                self.save_models()
        
        # Lưu mô hình cuối cùng
        self.save_models()
        print("Đã hoàn thành huấn luyện!")
    
    def get_actions(self, state):
        """
        Lấy hành động cho tất cả các robot dựa trên chính sách đã học
        """
        # Cập nhật vị trí robot và trạng thái gói hàng
        self.path_sharer.update_positions(state['robots'])
    
        # Thêm thông tin debug để hiểu rõ hơn về trạng thái packages
        print(f"State['packages'] trong get_actions: {state['packages']}")
        print(f"self.packages_status trước khi cập nhật: {self.packages_status}")
        
        # Cập nhật trạng thái các gói hàng
        self.update_package_status(state)
        print(f"self.packages_status sau khi cập nhật: {self.packages_status}")
        

        # print("Cập nhật trạng thái packages",state['packages'])
        
        # Dictionary lưu hành động di chuyển dự định
        intended_actions = {}
        
        for i, agent in enumerate(self.agents):
            robot = state['robots'][i]
            robot_pos = (robot[0] - 1, robot[1] - 1)
            
            # Lấy trạng thái hiện tại
            state_features = agent.get_state_representation(state, self.env, i)
            
            # Sử dụng Q-table để chọn hành động
            action_idx = agent.choose_action(state_features, epsilon_override=0)  # epsilon=0 để luôn khai thác
            intended_actions[i] = agent.idx_to_action[action_idx]
        
        # Giải quyết xung đột giữa các robot
        final_actions = self.path_sharer.resolve_conflicts(intended_actions)
        
        # Xác định hành động nhận/thả gói hàng
        package_actions = ['0'] * self.n_robots
        
        for i in range(self.n_robots):
            robot = state['robots'][i]
            carrying = robot[2]
            robot_pos = (robot[0] - 1, robot[1] - 1)
            
            if carrying > 0:
                # Robot đang mang gói hàng, kiểm tra xem có ở đích không
                for pkg in state['packages']:
                    if pkg[0] == carrying:
                        target_pos = (pkg[3] - 1, pkg[4] - 1)
                        if robot_pos == target_pos:
                            package_actions[i] = '2'  # Thả gói hàng
                        break
            else:
                # Robot không mang gói hàng, kiểm tra xem có gói hàng nào để nhận không
                for pkg in state['packages']:
                    pkg_id = pkg[0]
                    start_pos = (pkg[1] - 1, pkg[2] - 1)
                    
                    if robot_pos == start_pos and self.is_package_available(pkg_id):
                        package_actions[i] = '1'  # Nhận gói hàng
                        print(f"Robot {i} nhận gói {pkg_id} tại vị trí {start_pos}")
                        break
        
        # Kết hợp hành động di chuyển và hành động gói hàng
        return [(final_actions[i], package_actions[i]) for i in range(self.n_robots)]

# Train

In [6]:
# Hàm để huấn luyện mô hình
def train_model(env_file, n_robots=2, n_packages=2, n_episodes=20, max_steps=100):
    """
    Huấn luyện mô hình IQL cho các robot giao hàng
    
    Args:
        env_file: Đường dẫn đến file bản đồ
        n_robots: Số lượng robot
        n_packages: Số lượng gói hàng
        n_episodes: Số lượng episode huấn luyện
        max_steps: Số bước tối đa trong mỗi episode
    """
    env = Environment(env_file, max_time_steps=max_steps, n_robots=n_robots, n_packages=n_packages)
    
    # Khởi tạo mô hình
    model = MultiAgentDelivery(env, use_trained_model=False)
    
    # Huấn luyện mô hình
    model.train(n_episodes=n_episodes, max_steps=max_steps)
    
    # Lưu mô hình
    model.save_models()
    
    print("Đã hoàn thành huấn luyện và lưu mô hình!")

# Huấn luyện mô hình cho các bản đồ khác nhau
# train_model('map1.txt', n_robots=5, n_packages=100, n_episodes=300)
# train_model('map2.txt', n_robots=5, n_packages=100, n_episodes=300)
# train_model('map3.txt', n_robots=5, n_packages=500, n_episodes=200)
# train_model('map4.txt', n_robots=10, n_packages=500, n_episodes=200)
# train_model('map5.txt', n_robots=10, n_packages=1000, n_episodes=100)

# Sử dụng mô hình đã huấn luyện
class IQLDeliveryAgents:
    def __init__(self):
        """
        Lớp agent chính dùng để giao tiếp với môi trường
        """
        self.agents = []
        self.packages = []
        self.packages_free = []
        self.n_robots = 0
        self.state = None
        self.model = None
        self.is_init = False

    def init_agents(self, state):
        """
        Khởi tạo các agent dựa trên trạng thái ban đầu
        """
        self.state = state
        self.n_robots = len(state['robots'])
        
        # Khởi tạo môi trường và mô hình
        from env import Environment
        env = Environment(map_file='map5.txt')  # Truyền vào tên file bản đồ thích hợp
        env.grid = state['map']
        env.n_rows = len(env.grid)
        env.n_cols = len(env.grid[0]) if env.grid else 0
        env.robots = [None] * self.n_robots
        
        # Tạo mô hình IQL đã huấn luyện
        self.model = MultiAgentDelivery(env, use_trained_model=True)
        
    def get_actions(self, state):
        """
        Lấy hành động cho tất cả các robot dựa trên trạng thái hiện tại
        """
        if self.model is None:
            # Nếu chưa khởi tạo mô hình, hãy làm điều đó bây giờ
            self.init_agents(state)
            
        # Sử dụng mô hình để lấy hành động
        actions = self.model.get_actions(state)
        return actions

In [7]:
from env import Environment
import os
import numpy as np
import time

# Khởi tạo môi trường với tham số phù hợp
env_file = 'map5.txt'  # Bạn có thể chọn bản đồ từ map1.txt đến map5.txt
n_robots = 10          # Số lượng robot
n_packages = 300      # Số lượng gói hàng
max_steps = 1000       # Số bước tối đa mỗi episode
n_episodes = 1000      # Số lượng episode huấn luyện

env = Environment(map_file=env_file, max_time_steps=max_steps, 
                  n_robots=n_robots, n_packages=n_packages)

# Khởi tạo mô hình đa tác tử với tham số use_trained_model=False để huấn luyện từ đầu
model = MultiAgentDelivery(env, use_trained_model=False)

# Huấn luyện mô hình
print("Bắt đầu quá trình huấn luyện...")
model.train(n_episodes=n_episodes, max_steps=max_steps)

# Tạo thư mục models nếu chưa tồn tại
if not os.path.exists("models"):
    
    os.makedirs("models")
    
# Lưu mô hình
model.save_models()
print("Đã lưu mô hình vào thư mục 'models'")

Bắt đầu quá trình huấn luyện...


KeyboardInterrupt: 

# Test

In [9]:
# Khởi tạo môi trường mới để kiểm thử
test_env = Environment(map_file='map1.txt', max_time_steps=200, 
                      n_robots=5, n_packages=100)

# Sử dụng lớp IQLDeliveryAgents để kiểm thử
agents = IQLDeliveryAgents()
state = test_env.reset()
agents.init_agents(state)

# Hiển thị môi trường (có thể sử dụng VisualRenderer nếu muốn)
from visualize import VisualRenderer
renderer = VisualRenderer(cell_size=40)

try:
    renderer.load_images(images_dir='../images')
    print("Đã tải hình ảnh thành công")
except Exception as e:
    print(f"Lỗi tải hình ảnh: {e}")
    print("Sẽ sử dụng hình dạng mặc định")

# Chạy mô phỏng
done = False
total_reward = 0
step = 0

while not done and step < 500:
    # Lấy hành động từ mô hình
    # print(f"Trạng thái hiện tại: {state}")
    actions = agents.get_actions(state)
    
    print(f"Trạng thái hiện tại: {state}")
    #in ra các hành động
    print(f"Hành động dự kiến: {actions}")

    # Thực hiện hành động trên môi trường
    next_state, reward, done, info = test_env.step(actions)
    # print(f"Trạng thái: {state}, Hành động: {actions}, Phần thưởng: {reward}")
    # # Hiển thị trạng thái
    renderer.render(test_env)
    time.sleep(0.2)  # Tạm dừng để quan sát
    
    # Cập nhật trạng thái và thông tin
    state = next_state
    total_reward += reward
    step += 1
    print(f"Bước {step}, Phần thưởng: {reward}, Tổng phần thưởng: {total_reward}")

print(f"Kết thúc mô phỏng. Tổng phần thưởng: {total_reward}")

Đã tải Q-table cho agent 0 với 162768 trạng thái
Đã tải Q-table cho agent 1 với 220039 trạng thái
Đã tải Q-table cho agent 2 với 162768 trạng thái
Đã tải Q-table cho agent 3 với 220039 trạng thái
Đã tải Q-table cho agent 4 với 162768 trạng thái
Tìm hình ảnh tại: e:\Năm 3 Kỳ II\Học tăng cường\Cuối Kỳ\marl-delivery(new)\delivery-robots\marl-delivery-master\../images
Âm thanh được tải từ: e:\Năm 3 Kỳ II\Học tăng cường\Cuối Kỳ\marl-delivery(new)\delivery-robots\marl-delivery-master\../sounds\delivering.mp3
Đã tải hình ảnh thành công
State['packages'] trong get_actions: [(1, 7, 4, 8, 3, 0, 20), (2, 7, 6, 5, 3, 0, 27), (3, 9, 7, 6, 9, 0, 28), (4, 7, 9, 7, 5, 0, 25), (5, 8, 5, 4, 2, 0, 29), (6, 8, 3, 3, 8, 0, 19)]
self.packages_status trước khi cập nhật: {}
self.packages_status sau khi cập nhật: {1: 'waiting', 2: 'waiting', 3: 'waiting', 4: 'waiting', 5: 'waiting', 6: 'waiting'}
Trạng thái hiện tại: {'time_step': 0, 'map': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0

NameError: name 'exit' is not defined